In [1]:
# pricing an American put option - Longstaff-Schwarz
# option based on average price of two stocks 

import numpy as np 
import numpy.random as npr
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
# parameters of the stocks and the option 
S10 = 40.0
S20 = 40.0 
K = 40.0 
r = 0.06
sigma1 = 0.4
sigma2 = 0.2
rho = -0.5
rho_c = np.sqrt(1-rho**2) 
T = 2.0

In [3]:
# numeric parameters - number of intervals, number of samples, random numbers 
N = 250
h = T/N 
M = 50000

In [4]:
S1 = S10*np.ones([M,N+1])
S2 = S20*np.ones([M,N+1])
Z1 = npr.randn(M,N)
Z2 = npr.randn(M,N)
b = 3                    # number of basis functions
disc = np.exp(-r*h)      # one period discount factor 

In [5]:
# generate price paths using the GBM formula
for n in range(0,N):       
    S1[:,n+1]=S1[:,n]*np.exp((r-sigma1**2/2)*h+(Z1[:,n])                              *sigma1*np.sqrt(h))
    S2[:,n+1]=S2[:,n]*np.exp((r-sigma2**2/2)*h+(rho*Z1[:,n]+np.sqrt(1-rho**2)*Z2[:,n])*sigma2*np.sqrt(h)) 

In [6]:
# exercise value
def exer(S1,S2):
    return((K-(S1+S2)/2)*(K>(S1+S2)/2))

# Basis functions
def psi(i,S1,S2):
    if i==0:
        return(1)
    elif i==1:
        return(S1/K)
    elif i==2:
        return(S2/K)
    elif i==3:
        return(S1*S1/K*K)
    elif i==4:
        return(S1*S2/K*K)
    elif i==5:
        return(S2*S2/K*K)

In [7]:
c = np.zeros([b,N+1])     # coefficients of continuation values
d = np.zeros(b)
MM = np.zeros([b,b])      # normalization matrix
val = np.zeros([M,N+1])   # option values in simulation 
val[:,N] = exer(S1[:,N],S2[:,N])   # option values at final time 

In [8]:
for k in range(N-1,0,-1):
    sel = ( (S1[:,k]+S2[:,k])/2 < K )
    val[~sel,k] =  disc*val[~sel,k+1]
    S1sel = S1[sel,k]
    S2sel = S2[sel,k]
    valsel = disc*val[sel,k+1]
    for i in range(0,b):
        c[i,k] = np.mean( valsel *  psi(i,S1sel,S2sel))       # was called d in file          
        MM[i,i] = np.mean( psi(i,S1sel,S2sel) *  psi(i,S1sel,S2sel) )   
        for j in range(i+1,b):
            MM[i,j] = np.mean( psi(i,S1sel,S2sel) *  psi(j,S1sel,S2sel) ) 
            MM[j,i] = MM[i,j] 
    c[:,k] = np.linalg.solve(MM,c[:,k])
    cont = np.zeros(S1sel.size)
    for i in range(0,b):
        cont = cont + c[i,k] * psi(i,S1sel,S2sel)
    val[sel,k] = exer(S1sel,S2sel) * (exer(S1sel,S2sel) > cont) + disc*val[sel,k+1] * (exer(S1sel,S2sel) < cont)
val[:,0] =  disc*val[:,1]
V1=np.mean(val[:,0])
e1=np.std(val[:,0])/np.sqrt(M)
print("price of the option is: {:.3f}, with a stochastic error of {:.5f}".format(V1,e1))

price of the option is: 2.291, with a stochastic error of 0.01229


In [9]:
# generate a further set of paths to get lower bound on option price 
M2 = 50000
Sq1 = S10*np.ones([M2,N+1])
Sq2 = S20*np.ones([M2,N+1])
Zq1 = npr.randn(M2,N)
Zq2 = npr.randn(M2,N)
for n in range(0,N): 
    tem1 =(r-sigma1**2/2)*h + np.sqrt(h)*sigma1*Zq1[:,n]
    tem2 =(r-sigma2**2/2)*h + np.sqrt(h)*sigma2*(rho*Zq1[:,n] + rho_c*Zq2[:,n])      
    Sq1[:,n+1]=Sq1[:,n]*np.exp(tem1 )
    Sq2[:,n+1]=Sq2[:,n]*np.exp(tem2)    
val2 = np.zeros(M2)
for n in range(1,N+1): 
    tem1 = exer( Sq1[:,n], Sq2[:,n] )
    tem2 = np.zeros(M2)
    for i in range(0,b):
        tem2 = tem2 + c[i,n] * psi(i,Sq1[:,n],Sq2[:,n])
    val2 = val2 + np.exp(-r*n*h)*(val2==0)*(tem1>tem2)*tem1
V2=np.mean(val2)
e2=np.std(val2)/np.sqrt(M2)
print("price of the option is: {:.3f}, with a stochastic error of {:.5f}".format(V2,e2))

price of the option is: 2.286, with a stochastic error of 0.01228
